In [4]:
from pymongo import MongoClient
from pprint import pprint
import json

client = MongoClient("localhost", 27017)
client.drop_database("userData")

db = client.userData
with open("people.json") as f:
    file_data = json.load(f)
db.users.insert_many(file_data)


In [5]:
for x in db.users.aggregate(
    [
        {
            "$bucket": {
                "groupBy": "$dob.age",
                "boundaries": [18, 30, 40, 50, 60, 80],
                "output": {
                    "num_people": {"$sum": 1},
                    "age_avg": {"$avg": "$dob.age"},
                },
            }
        },
        {"$limit": 5},
    ]
):
    pprint(x)


{'_id': 18, 'age_avg': 25.101382488479263, 'num_people': 868}
{'_id': 30, 'age_avg': 34.51758241758242, 'num_people': 910}
{'_id': 40, 'age_avg': 44.42265795206972, 'num_people': 918}
{'_id': 50, 'age_avg': 54.533811475409834, 'num_people': 976}
{'_id': 60, 'age_avg': 66.55798192771084, 'num_people': 1328}


In [6]:
for x in db.users.aggregate(
    [
        {
            "$bucketAuto": {
                "groupBy": "$dob.age",
                "buckets": 5,
                "output": {
                    "num_people": {"$sum": 1},
                    "age_avg": {"$avg": "$dob.age"},
                },
            }
        },
        {"$limit": 5},
    ]
):
    pprint(x)


{'_id': {'max': 32, 'min': 21},
 'age_avg': 25.99616122840691,
 'num_people': 1042}
{'_id': {'max': 43, 'min': 32},
 'age_avg': 36.97722772277228,
 'num_people': 1010}
{'_id': {'max': 54, 'min': 43},
 'age_avg': 47.98838334946757,
 'num_people': 1033}
{'_id': {'max': 65, 'min': 54},
 'age_avg': 58.99342105263158,
 'num_people': 1064}
{'_id': {'max': 74, 'min': 65}, 'age_avg': 69.11515863689776, 'num_people': 851}


In [7]:
client.close()
